In [62]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity




In [63]:

movies_df = pd.read_csv("movies.csv")
# skmetric.pairwise.cosine_similarity(X, Y=None, dense_output=True)[source]


In [64]:
movies_df.head()

,movieId,genres,id,imdb_id,original_language,original_title,overview,popularity,release_date,runtime,spoken_languages,title,vote_average,vote_count,rating,genre
0,1,"['Adventure', 'Fantasy', 'Family']",8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15 00:00:00,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji,6.9,2413.0,3.89,"Adventure,Fantasy,Family"
1,2,"['Romance', 'Comedy']",15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,1995-12-22 00:00:00,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men,6.5,92.0,3.24,"Romance,Comedy"
2,3,"['Comedy', 'Drama', 'Romance']",31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22 00:00:00,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Waiting to Exhale,6.1,34.0,3.18,"Comedy,Drama,Romance"
3,4,['Comedy'],11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,1995-02-10 00:00:00,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Father of the Bride Part II,5.7,173.0,2.88,Comedy
4,5,"['Action', 'Crime', 'Drama', 'Thriller']",949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.924927,1995-12-15 00:00:00,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Heat,7.7,1886.0,3.08,"Action,Crime,Drama,Thriller"


In [65]:
movies_df.dropna(inplace=True)

1. Features which represent the global averages
The three global averages we’ll employ are:

- The average ratings of all movies given by all users
- The average ratings of a particular movie given by all users
- The average ratings of all movies given by a particular user

In [66]:
ratings = pd.read_csv("ratings.csv")

In [79]:
ratings

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0
...,...,...,...
26024284,270896,58559,5.0
26024285,270896,60069,5.0
26024286,270896,63082,4.5
26024287,270896,64957,4.5


Convert the genre col to binary(get dummies)


In [67]:
genres_list=[]
for ls in movies_df['genre'].str.split(","):
    for genre in ls:
        
        if genre not in genres_list:
             genres_list.append(genre)

    


In [68]:
#Creating a dataframe out of the genres so I can add them to my movies dataframe 
genres_df = pd.DataFrame({'genres': genres_list})

In [69]:
genres_df

,genres
0,Adventure
1,Fantasy
2,Family
3,Romance
4,Comedy
5,Drama
6,Action
7,Crime
8,Thriller
9,Horror


In [70]:
unique_genres = set(','.join(genres_df['genres']).split(','))


In [71]:
unique_genres

{'Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Foreign',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western'}

In [72]:

for genre in unique_genres:
    movies_df[genre] = movies_df['genres'].str.contains(genre).astype(int)

movies_df.drop('genres',inplace=True,axis=1)

Add a new column name `liked` for the users who rated the movie above 3.5

In [80]:
ratings['liked'] = ratings['rating'].apply(lambda x: 1 if x >= 3.5 else 0)

,userId,movieId,rating,liked
0,1,110,1.0,0
1,1,147,4.5,1
2,1,858,5.0,1
3,1,1221,5.0,1
4,1,1246,5.0,1
...,...,...,...,...
26024284,270896,58559,5.0,1
26024285,270896,60069,5.0,1
26024286,270896,63082,4.5,1
26024287,270896,64957,4.5,1




 more common a term: lower the idf(higher weight for less common wrods.)

 
 sort the tfidf values: highest tfidf scores would be our doc profile.

In [73]:
tfVec = TfidfVectorizer(analyzer = 'word' , stop_words='english',min_df=0,ngram_range=(1,2))

In [74]:
tfVec_matrix = tfVec.fit_transform(movies_df['genre'])

In [75]:
#sanity check on the matrix
tfVec_matrix.shape

(10691, 343)

In [76]:
cosine_matrix = cosine_similarity(tfVec_matrix,tfVec_matrix)

In [78]:
#User's preffered genres:
preffered_genres =['Animation','Comedy']
# Sort the genres based on the cosine similarity scores
similar_genres = cosine_matrix[preffered_genres].sum().sort_values(ascending=False).head(5)

# Get the recommended movies based on the similar genres
recommended_movies = movies_df[movies_df['genre'].apply(lambda x: any(genre in x for genre in similar_genres.index))]



IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

Writing a function to return the movies which are comon between two users(common_movies)

In [109]:
ratings[ratings['movieId']==110]['userId']

0                1
417             11
1661            22
1733            24
2491            29
             ...  
26020849    270885
26023563    270891
26023676    270893
26024010    270895
26024040    270896
Name: userId, Length: 66512, dtype: int64

In [125]:
def common_movies(user1,user2):
    movies=[]
    movies_id = [movie for movie in ratings[ratings['userId']==user1]['movieId'] if movie in ratings[ratings['userId']==user2]['movieId']]
    for id in movies_id:
        movies.append(movies_df[movies_df['movieId'] == id]['title']) 
    return movies
print(common_movies(22,11))    


[453    A Good Man in Africa
Name: title, dtype: object, 462    The Hudsucker Proxy
Name: title, dtype: object, 539    Terminal Velocity
Name: title, dtype: object, 548    The Pagemaster
Name: title, dtype: object]


In [168]:
def get_common_liked_movie(user1,user2):
    liked_movies=[]
    liked_movies_id = [movie for movie in common_movies(user1,user2) if (ratings[(ratings['userId']==user1) & (ratings['liked']==1)]) & \
                       (ratings[(ratings['userId']==user2) & (ratings['liked']==0)]) ]
    for id in liked_movies_id:
        print(id)
        # liked_movies.append(movies_df[movies_df['movieId'] == id]['title']) 
    # return liked_movies

In [171]:
ratings[(ratings['userId']==2) & (ratings['liked']==0)]

,userId,movieId,rating,liked
27,2,5,3.0,0
28,2,25,3.0,0
29,2,32,2.0,0
30,2,58,3.0,0
33,2,141,3.0,0
40,2,762,3.0,0
41,2,780,3.0,0
42,2,786,1.0,0
43,2,788,1.0,0
47,2,1475,3.0,0


In [169]:
print(common_movies(22,11))  

[453    A Good Man in Africa
Name: title, dtype: object, 462    The Hudsucker Proxy
Name: title, dtype: object, 539    Terminal Velocity
Name: title, dtype: object, 548    The Pagemaster
Name: title, dtype: object]
